# Read Data

In [1]:
from warnings import filterwarnings
filterwarnings("ignore")

In [2]:
import pandas as pd

In [3]:
A=pd.read_csv("C:/Users/Suja4/OneDrive/Documents/Data Science/Dataset/Credit.csv")

In [4]:
A.head()

,Unnamed: 0,ID,Income,Limit,Rating,Cards,Age,Education,Gender,Student,Married,Ethnicity,Balance
0,1,1,14.891,3606,283,2,34,11,Male,No,Yes,Caucasian,333
1,2,2,106.025,6645,483,3,82,15,Female,Yes,Yes,Asian,903
2,3,3,104.593,7075,514,4,71,11,Male,No,No,Asian,580
3,4,4,148.924,9504,681,3,36,11,Female,No,No,Asian,964
4,5,5,55.882,4897,357,2,68,16,Male,No,Yes,Caucasian,331


# Remove null values 

In [5]:
A.isna().sum()

Unnamed: 0    0
ID            0
Income        0
Limit         0
Rating        0
Cards         0
Age           0
Education     0
Gender        0
Student       0
Married       0
Ethnicity     0
Balance       0
dtype: int64

# Outliers

In [6]:
from Wd8pm import OL
outliers=OL(A)

In [7]:
outliers

[28, 85, 184, 205, 209, 261, 275, 323, 347, 355, 383]

In [8]:
A=A.drop(index=outliers,axis=0)


In [9]:
A.shape

(389, 13)

In [10]:
A.index=range(0,389)

# Find unique columns

In [11]:
A.nunique()

Unnamed: 0    389
ID            389
Income        388
Limit         376
Rating        274
Cards           7
Age            66
Education      16
Gender          2
Student         2
Married         2
Ethnicity       3
Balance       276
dtype: int64

# Define X and Y

In [12]:
Y= A[['Student']]
X=A.drop(labels=['Unnamed: 0','ID','Student'],axis=1)

# Bivariate analysis

In [32]:
from Wd8pm import ANOVA ,chisq

for i in X.columns:
    if (X[i].dtypes=='object'):
        pval=chisq(A,"Student",i)
        print("Student ~",i,"--Chisq-->",pval)
    else:
        pval=ANOVA(A,"Student",i)
        print("Student ~",i,"--Anova-->",pval)
      
        

Student ~ Income --Anova--> 0.76899
Student ~ Limit --Anova--> 0.95976
Student ~ Rating --Anova--> 0.969
Student ~ Cards --Anova--> 0.64699
Student ~ Age --Anova--> 0.50861
Student ~ Education --Anova--> 0.08654
Student ~ Gender --Chisq--> 0.4276
Student ~ Married --Chisq--> 0.15047
Student ~ Ethnicity --Chisq--> 0.68199
Student ~ Balance --Anova--> 0.0


In [28]:
for i in X.columns:
    print(i)

Income
Limit
Rating
Cards
Age
Education
Gender
Married
Ethnicity
Balance


In [29]:
X.columns

Index(['Income', 'Limit', 'Rating', 'Cards', 'Age', 'Education', 'Gender',
       'Married', 'Ethnicity', 'Balance'],
      dtype='object')

# Preprocessing 

In [14]:
Ximp = X[["Balance","Education","Married"]]
from Wd8pm import preprocessing
Xnew =preprocessing(Ximp)

# Split

In [15]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(Xnew,Y,test_size=.2,random_state=21)

#    # Model

In [16]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
model=lr.fit(xtrain,ytrain)

# Prediction

In [17]:
tr_pred =model.predict(xtrain)
ts_pred =model.predict(xtest)

# Evaluation

In [18]:
from sklearn.metrics import accuracy_score
accuracy_score(ytrain,tr_pred) ,accuracy_score(ytest,ts_pred)

(0.9003215434083601, 0.9102564102564102)

In [19]:
from sklearn.metrics import confusion_matrix,recall_score
confusion_matrix(ytrain,tr_pred)

array([[277,   2],
       [ 29,   3]], dtype=int64)

In [20]:
confusion_matrix(ytest,ts_pred)

array([[71,  0],
       [ 7,  0]], dtype=int64)

# Data

In [21]:
xtest['pred']=ts_pred

In [22]:
xtest.index

Int64Index([ 20,  89, 130, 209,  59,  37, 362, 106, 192, 141, 212,  72, 270,
            349, 367,  92, 201, 127, 369, 228,  51, 370, 164,  52, 119, 232,
            352,  55,  11, 365,  40, 251, 191,  47, 323, 331,  96, 198, 316,
             91, 387, 178, 174, 185, 259, 168, 319, 333,   9, 195,  61,  66,
            136, 180, 104, 216, 213, 377, 159,  16, 366, 275, 276, 134,  70,
             45, 224, 219, 310, 190, 176, 318, 263, 354,  93, 196,   6, 382],
           dtype='int64')

In [23]:
Xdata = Ximp.iloc[xtest.index,:]

In [24]:
Xdata['pred']=ts_pred

In [25]:
Xdata

,Balance,Education,Married,pred
20,89,16,No,No
89,812,16,Yes,No
130,912,13,Yes,No
209,681,16,Yes,No
59,631,20,No,No
...,...,...,...,...
354,1243,16,Yes,No
93,0,17,Yes,No
196,571,7,Yes,No
6,203,12,No,No


In [26]:
Xdata['Y'] = Y.iloc[xtest.index]

In [27]:
Xdata

,Balance,Education,Married,pred,Y
20,89,16,No,No,No
89,812,16,Yes,No,No
130,912,13,Yes,No,No
209,681,16,Yes,No,No
59,631,20,No,No,No
...,...,...,...,...,...
354,1243,16,Yes,No,No
93,0,17,Yes,No,No
196,571,7,Yes,No,No
6,203,12,No,No,No
